In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import mediapipe as mp
import cv2
import time
import matplotlib.pyplot as plt
import os
import csv

2022-08-27 04:55:14.260593: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-27 04:55:14.260625: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils 

In [3]:
def mediapipe_detection(image, model):
    if image is not None:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
        image.flags.writeable = False                  
        results = model.process(image)                 
        image.flags.writeable = True                   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
        return image, results

In [32]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
      
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [4]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [5]:
DATA_PATH = os.path.join('MP_Data9') 

no_sequences = 60

sequence_length = 40

f = open('actions.csv', 'r')
reader = csv.reader(f)
actions = []
for word in reader:
    actions.append(word[0])
print(actions)
n = len(actions)

['Thank you', 'ATM', 'Idle', 'Hi', 'My', 'Loan', 'Account']


In [35]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

ModuleNotFoundError: No module named 'sklearn'

In [36]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
y_test.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Bidirectional,GRU,Dropout,Conv1D,MaxPooling1D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.regularizers import L2

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(Conv1D(128, 3, activation='relu',input_shape=(40,1662)))
model.add(Conv1D(256, 3, activation='relu'))
model.add(MaxPooling1D(pool_size=(2), strides=(2)))
model.add(Dropout(0.10))
model.add(Dense(30, activation='relu'))
model.add(LSTM(256, return_sequences=True, activation='tanh'))
model.add(LSTM(256, return_sequences=False, activation='tanh'))
model.add(Dense(256, activation='relu', kernel_regularizer=L2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.40))
model.add(Dense(n, activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=50, callbacks=[tb_callback])

In [6]:
model = tf.keras.models.load_model('conv-rnn-model.h5')

2022-08-27 04:55:37.179674: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-27 04:55:37.179712: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2022-08-27 04:55:37.179988: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
sequence = []
sentence = []
threshold = 0.90

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.77, min_tracking_confidence=0.77) as holistic:
    while cap.isOpened():

        
        ret, frame = cap.read()

        if frame is not None:
            image, results = mediapipe_detection(frame, holistic)

            draw_styled_landmarks(image, results)
        

            keypoints = extract_keypoints(results)

            sequence.append(keypoints)
            sequence = sequence[-sequence_length:]
        
            if len(sequence) == sequence_length:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
            
  
                if res[np.argmax(res)] > threshold: 

                    tmp = actions[np.argmax(res)]
                    if(len(sentence) > 0):
                        if(not(sentence[-1] == tmp)):
                            sentence.append(tmp)
                    else:
                        sentence.append(tmp)
                    if(len(sentence) > 1):
                        
                        if(tmp != 'Idle'  and sentence[-2] == 'Idle'):
                            del sentence[-2]
                        elif(tmp == 'Idle'):
                            sentence = []
                                
                         
                if len(sentence) > 3: 
                    sentence = sentence[-1:]

            cv2.putText(image, ' '.join(sentence), (3,85), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,235), 2, cv2.LINE_AA)
        
            cv2.imshow('OpenCV Feed', image)
                                  
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


: 

: 

In [ ]:
model.save('conv-rnn-model.h5')